In [1]:
#Author-Vaishnavi Sabhahita
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier

In [2]:
print('start')
promoted_content = pd.read_csv('all/promoted_content.csv')
br=50000
train = pd.read_csv('all/clicks_train.csv',iterator=True,chunksize=br)
print('done')

start
done


In [3]:
for br1 in train:
    br1=pd.merge(br1,promoted_content,how='left',on='ad_id')
    predictors=[x for x in br1.columns if x not in ['display_id','clicked']]
    #print(predictors) #['ad_id', 'document_id', 'campaign_id', 'advertiser_id']
    br1=br1.fillna(0.0)
    alg = RandomForestClassifier(random_state=1, n_estimators=3, min_samples_split=4, min_samples_leaf=2, warm_start=True)
    alg.fit(br1[predictors], br1["clicked"])
print('training done')

training done


In [4]:
train=''
test= pd.read_csv('all/clicks_test.csv',iterator=True,chunksize=br) #Load data
prediction=[]
print('test loaded')

test loaded


In [5]:
for br1 in test:
    init_br_size=len(br1)
    br1=pd.merge(br1,promoted_content,how='left',on='ad_id')
    br1=br1.fillna(0.0)
    br_pred=list(alg.predict_proba(br1[predictors]).astype(float)[:,1])
    prediction += br_pred
print('prediction')

prediction


In [6]:
test=''
test= pd.read_csv('all/clicks_test.csv')
print('done')

done


In [7]:
finale=pd.concat((test,pd.DataFrame(prediction)) ,axis=1,ignore_index=True)
print(finale.head(10))
print('results')

          0       1         2
0  16874594   66758  0.200000
1  16874594  150083  0.133333
2  16874594  162754  0.150697
3  16874594  170392  0.133333
4  16874594  172888  0.723810
5  16874594  180797  0.266667
6  16874595    8846  0.047619
7  16874595   30609  0.000000
8  16874595  143982  0.000000
9  16874596   11430  0.233333
results


In [8]:
finale.columns = ['display_id','ad_id','clicked']
finale.sort_values(['display_id','clicked'], inplace=True, ascending=False)
finale = finale.groupby('display_id').ad_id.apply(lambda x: " ".join(map(str,x))).reset_index()
finale.to_csv('all/final.csv', index=False)
print('output stored')

output stored
